In [34]:
# data and labels uncut -> 72%
# data and labels without the class no birds -> 52% looks like its bad with classifiying birds
# data and labels without clean data -> 74%
# data and labels with non redundant features -> 79%
# data and labels without clean data and with non redundant features -> 76%
# data and labels with non redundant features and use 8 layers instead of 7-> 78% almost the same as 7 layers gonna stick w 7
# data and labels with non redundant features and no birds -> 61 %
# data and labels with non redundant features and classify bird exist or not -> 88%
# data and labels with non redundant features and nobirds dropped till largest group of birds -> 77%
# data and labels with non redundant features and classify bird exist or not and 6 layers-> 88%
# new data split uncut -> 53%
# new data split with non redundant features -> 61 %
# new data split with non redundant features and normalized -> 89 %
# data split with non redundant features and normalized -> 84 %
# new data split with non redundant features threshold 0.75 -> 88%
# new data split with non redundant features threshold 0.8 and robust normalization -> 96.4%

In [57]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [58]:
data = pd.read_csv('./dataset/new/new_data.csv').drop(columns="Unnamed: 0")
labels = pd.DataFrame(pd.read_csv('./dataset/new/new_labels.csv')["Aggregated"])

In [55]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# for fold, (train_index, test_index) in enumerate(kf.split(data)):
#     print(f"Fold {fold}")
#     X_train, X_test = data.iloc[train_index], data.iloc[test_index]
#     y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

#     # Convert data and labels to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
#     X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
#     y_test_tensor = torch.tensor(y_test.values, dtype=torch.int64)

# train_data = []
# train_label = []
# for i in range(0,7):
#     index = labels[labels == 1].sample(2324).index
#     for j in range(2324):
#         train_data.append(data.iloc[index[j]].to_numpy())
#         train_label.append(i)

# train_data = np.array(train_data)
# train_label = np.array(train_label)

In [59]:
class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 16)
        self.fc7 = nn.Linear(16, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.relu(x)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.fc7(x)
        x = self.softmax(x)
        return x

In [60]:
# Define the hyperparameters
input_dim = 548
hidden_dim = 64
output_dim = 7
lr = 0.0001
batch_size = 64
num_epochs = 10

In [61]:
correlation_matrix = data.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
reduced_matrix = correlation_matrix.mask(mask)
feature_redundancy = reduced_matrix.abs().sum()
sorted_features = feature_redundancy.sort_values(ascending=False)

threshold = 0.8
selected_features = []
for feature in sorted_features.index:
    if not any(np.abs(correlation_matrix[feature][selected_features]) > threshold):
        selected_features.append(feature)

data = data[selected_features]

In [62]:
from sklearn.preprocessing import RobustScaler

# Initialize the RobustScaler
scaler = RobustScaler()

# Apply Robust Scaling to the DataFrame
scaled_data = scaler.fit_transform(data)

# Convert the scaled data back to a DataFrame
data = pd.DataFrame(scaled_data, columns=data.columns)

In [63]:
# Convert data and targets to numpy arrays
input_dim = len(data.columns)
data = data.to_numpy()
targets = labels.to_numpy().squeeze()

In [64]:
# Define the cross-validation method
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

# Loop over the folds
fold_accuracies = []
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f'Fold {fold_idx+1}')

    # Split the data into training and validation sets
    train_data, train_targets = data[train_idx], targets[train_idx]
    val_data, val_targets = data[val_idx], targets[val_idx]

    # Create the dataloaders for training and validation
    train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float32), torch.tensor(train_targets, dtype=torch.long))
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(torch.tensor(val_data, dtype=torch.float32), torch.tensor(val_targets, dtype=torch.long))
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    # Create the model, optimizer and loss function
    model = MyModel(input_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Train the model
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_dataloader):
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if i % 1000 == 999:    # Print every 1000 mini-batches
                print(f'Epoch {epoch+1}, Batch {i+1}: loss = {running_loss/1000:.3f}')
                running_loss = 0.0

    # Evaluate the model on the validation set
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_dataloader)
    accuracy = correct / total

    fold_accuracies.append(accuracy)
    print(f"Fold {fold_idx+1}, Validation Loss: {val_loss:.4f} Accuracy: {accuracy:.4f}")

# Print the average accuracy
print(f"Total acc: {np.array(fold_accuracies).mean()}")

Fold 1
Fold 1, Validation Loss: 1.1997 Accuracy: 0.9668
Fold 2
Fold 2, Validation Loss: 1.1976 Accuracy: 0.9674
Fold 3
Fold 3, Validation Loss: 1.1997 Accuracy: 0.9652
Fold 4
Fold 4, Validation Loss: 1.2090 Accuracy: 0.9567
Fold 5
Fold 5, Validation Loss: 1.2023 Accuracy: 0.9639
Total acc: 0.9639922231154348


In [65]:
fold_accuracies


[0.966796875,
 0.9674479166666666,
 0.9651692708333334,
 0.9566916313904266,
 0.963855421686747]

In [66]:
print(f"Total acc: {np.array(fold_accuracies).mean()}")

Total acc: 0.9639922231154348
